In [1]:
path = %pwd
import sys
sys.path.append(path + '/../')

In [2]:
from model import Mistral
from modules import MoeArgs, ModelArgs, MistralBlock
import torch

In [3]:
config = ModelArgs(
    model_type = None,
    cxt_size   = 20,
    embd_dim   = 32,
    n_layer    = 3,
    head_dim   = 32,
    hidden_dim = 128,
    n_head     = 8,
    n_kv_head  = 2,
    norm_eps   = 1e-5,
    vocab_size = 20,
    p_drop     = 0.1,
    rope_theta = 1e4,
)

In [4]:
model = Mistral(config)
model.n_params

99168

In [5]:
block = model.transformer.h[0]

In [6]:
freq_cis = model.freq_cis
freq_cis.shape

torch.Size([20, 16])

## Test block with inputs of various sizes

In [7]:
full_x = torch.randn(1, config.cxt_size, config.embd_dim)
full_y = block(full_x, freq_cis)
full_y.shape

torch.Size([1, 20, 32])

In [8]:
t = 8
part_x = torch.randn(1, t, config.embd_dim)
_freq_cis = freq_cis[:t, :]
part_y = block(part_x, _freq_cis)
part_y.shape

torch.Size([1, 8, 32])

## Test model with inputs of various sizes

In [9]:
full_x = torch.randint(0, config.vocab_size-1, (1, config.cxt_size))
full_y = model(full_x)
full_y.shape

torch.Size([1, 20, 20])

In [12]:
t=9
part_x = torch.randint(0, config.vocab_size-1, (1, t))
part_y = model(part_x)
part_y.shape

torch.Size([1, 9, 20])